# What is Recurring in Maths ?
- Recurring in Maths is simply the repettition of a decimal or an expression.
- A recurrence relation is an equation that defines a sequence based on a rule that gives the next term as a function of the previous term(s).
- The simplest form of a recurrence relation is the case where the next term depends only on the immediately previous term.

# Recurrent Neural Networks(RNN) in Deep Learning.
- Recurrent Neural Networks (RNN) are a class of Artificial Neural Networks that can process a sequence of inputs in deep learning and retain its state while processing the next sequence of inputs.
- Traditional neural networks will process an input and move onto the next one disregarding its sequence.

# **How RNN works:**
- <img src="https://www.simplilearn.com/ice9/free_resources_article_thumb/Fully_connected_Recurrent_Neural_Network.gif" width="800" height="350">
- It uses previous information to affect later ones.
-There are 3 layers: **Input(x), Hidden(h) and output(y).**
- In forward step when an input is recieved into the cell it gets multiplied by an **initialized weight** and then it is been **sent as an output** and then **used as an input in the next neuron**.
- Let us visualize it using a cell level diagram.
- <img src="https://blog.floydhub.com/content/images/2019/06/ezgif.com-video-to-gif.gif" width="500" height="350">
- The output of other neurons gets weight initialization as well. Except the first neuron of each layer all other neurons get inputs from **two ends 1) from the previous neuron and 2) other as an input** i.e., next element in the data.
- This is how any current neuron will have info of its current and previous data simultaneously.
- *The loop*: passes the input forward sequentialy, while retaining information about it.
- Here the information possesed by a neuron is passed on to the next one as well so that the new neuron will learn new pattern by keeping previous pattern in mind.
- This is how the sequential information in the data is preserved.
- Thus, It performs very good when it is given any sequential data of numeric or textual format.

# Types of RNN
- There are four types of RNNs
- <img src="https://static.packt-cdn.com/products/9781789536089/graphics/b2e068f5-08f8-4e4a-b56c-e29675ab0eb5.png" width="1100" height="400">

# Understanding the Maths behind RNN
- Let us follow up with this diagram of **Many to Many RNN** to understand more.
- <img src="https://miro.medium.com/max/2967/1*7_pAvVIMNp8h2aI4sdWdLg.png" width="800" height="350">
- Here **a0** is the initialized hidden state, It is being initialized because when start of there is no hidden state available for the first hidden cell.
- There is also an input **x0** comming into the the cell. Now both of this gets their own **weights initialized** and the hidden state becomes **Wha.a0** and the input becomes **Whx.x0**.
- Both of this dot products gets added and becomes **(Wha.a0+Whx.x0)**. Now after adding a **Bais(bh)** vector the whole sum becomes **(Wha.a0+Whx.x0+bh)**.
- Now this goes into an Activation function, **TanH** is most commonly used Activation function in RNNs as it keeps the range of values in between **range(-1,1)**.
- Now the output is **F(Wha.a0+Whx.x0+bh)**, where **F** is Tanh activation function.
- Let us visualize the whole above process in below GIF.
- <img src="https://miro.medium.com/max/1400/1*WMnFSJHzOloFlJHU6fVN-g.gif" width="800" height="350">
- In above gif, the **new hidden state** is the output of the current hidden cell which can be regarded as **F(Wha.a0+Whx.x0+bh)** from the above math.
- Now in the new hidden cell, this output recieved acts as an hidden state.
- Same can be visualized in below gif.
- <img src="https://miro.medium.com/max/1900/1*o-Cq5U8-tfa1_ve2Pf3nfg.gif" width="900" height="300">
- This is how RNNs are different from the ANNs, Thats all about RNNs let us proceed further to see its implementation.

- **NOTE:** Please refer to a more proper illustration about how RNN works before proceeding any further. The above ones are just simple tips about how different they are from the Artificial Neural Networks.
- For Further info please go through this awesome blog:
[RNN by Stanford](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks), and for video watch this simple illustration [RNN Illustration Video](https://www.youtube.com/watch?v=LHXXI4-IEns&t=496s&ab_channel=TheA.I.Hacker-MichaelPhiTheA.I.Hacker-MichaelPhi)

# Importing Packages

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Let us Build a RNN

## Setting up **DATASET** and **HYPERPARAMETERS**.

In [ ]:
input_size = 28 # Image of 28x28 pixels which upon flattening becomes 784.
sequence_length = 28
num_of_layers = 2 # Number of RNN layers we want.
hidden_size = 256 # Number of neuron per hidden state.
num_classes = 10 # Number of classes in the Dataset are 10.
learning_rate = 0.001 # Speed at which we want our optimizer to optimize our solution.
batch_size = 64 # Size of the batch that will undergo training at one step.
epochs = 2 # Steps of training or times a forward and backward propagation is done.

# Firstly Loading a Data and downloading it to the folder.
train_dataset = datasets.MNIST(root="content/",train=True,
                               transform=transforms.ToTensor(),download=True)
# Now setting up its properties like batchsize
trainloader = DataLoader(dataset=train_dataset,
                         batch_size = batch_size,
                         shuffle=True
                         )

# Doing the same for testset as well
test_dataset = datasets.MNIST(root="content/",train=False,
                               transform=transforms.ToTensor(),download=True)

testloader = DataLoader(dataset=test_dataset,
                         batch_size = batch_size,
                         shuffle=True
                         )

## Creating RNN 

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_of_layers, num_of_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_of_layers = num_of_layers
    self.num_of_classes = num_of_classes
    self.rnn = nn.RNN(input_size, hidden_size, num_of_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_of_classes)
  
  def forward(self, x):
    # Initializing hidden state.
    h0 = torch.zeros(self.num_of_layers, x.size(0), self.hidden_size) # Initial Hidden State for each element in the batch.
    # Forward step.
    rnn_out_vector,_ = self.rnn(x, h0) # It takes in input vector followed by hidden state, returns and output vector along with hidden state.
    rnn_out_vector = rnn_out_vector.reshape(rnn_out_vector.shape[0],-1)
    output_from_linear_layer = self.fc(rnn_out_vector) # Sending in output of RNN into the Linear Layer(ANN)
    return output_from_linear_layer


# Initializing Model, Loss, Optimizer

In [ ]:
# Initializing the Model
model = RNN(input_size, hidden_size, num_of_layers, num_classes)

In [ ]:

# Using CrossEntropyLoss as we have multiple classes.
loss_function = nn.CrossEntropyLoss()

# Optimizer as ADAM
optimizer = optim.Adam(params=model.parameters(),lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  for batch_index, (data, targets) in enumerate(trainloader):# output of format = batch_index, (element_data, its_target)
    data=data.squeeze(1)
    # Forward Step
     # Making Predictions on train_data
    training_predictions = model(data)
     # Calculating loss
    Training_loss = loss_function(training_predictions, targets)
    # Backward Step
    optimizer.zero_grad()
    Training_loss.backward()

    # Optimizer Step
    optimizer.step()
  print(f'At {epoch} epochs Training_loss={Training_loss}')



At 0 epochs Training_loss=0.0071024359203875065
At 1 epochs Training_loss=0.2920049726963043


In [ ]:
# Custom Function that calcultes accuracy of the Model.

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.squeeze(1)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Got {num_correct} / {num_samples} with accuracy = {float(num_correct)/float(num_samples)*100:.2f}")

In [ ]:
model.train() # Training the Model once again.
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Checking accuracy on training data
Got 57591 / 60000 with accuracy = 95.98
Checking accuracy on test data
Got 9608 / 10000 with accuracy = 96.08


- Well thats some good score. So this was all about the basic RNN Implementation.
# **THANK YOU!**